In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
%matplotlib inline

#import sys
#sys.setrecursionlimit(20000)

import edward as ed
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap, Normalize
from matplotlib.animation import FuncAnimation
from tempfile import NamedTemporaryFile
from IPython.display import HTML
import seaborn as sns
import numpy as np
import six
import tensorflow as tf

#################################
import tensorflow_probability
import keras
#import edward2##################

plt.style.use('seaborn-talk')
sns.set_context("talk", font_scale=1.4)
sns.set_palette("colorblind")
sess = ed.get_session()
#sess = tf.compat.v1.Session()###########

sns.palplot(sns.color_palette())

In [ ]:
# this can be done only before using Edward
ed.set_seed(42)

In [ ]:
VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)

In [ ]:
def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))

# Coin

In [ ]:
from edward.models import Bernoulli, Beta, Empirical, Uniform # import distributions

In [ ]:
N = 100  # sample size

In [ ]:
def build_fair_dataset(N): # the results of 100 flips for a fair coin 
    pheads = tf.constant(0.5)
    c = Bernoulli(probs=pheads, sample_shape=N)
    return sess.run([pheads, c])

In [ ]:
def build_unfair_dataset(N): # the results of 100 flips for an unfair coin 
    pheads = tf.constant(0.05)
    c = Bernoulli(probs=pheads, sample_shape=N)
    return sess.run([pheads, c])

In [ ]:
def build_dataset(N):
    pheads = Uniform(low=0.0, high=1.0)    # get a random probability of getting heads
    c = Bernoulli(probs=pheads, sample_shape=N)  # get a sample with the probability of getting heads = pheads
    return sess.run([pheads, c])

In [ ]:
# DATA
pheads_true, c_train = build_fair_dataset(N)

In [ ]:
pheads_true

In [ ]:
c_train

In [ ]:
sum(c_train == 0)

In [ ]:
sum(c_train == 1)

In [ ]:
pheads_fair = Beta(concentration1=1000.0, concentration0=1000.0)  # blue
pheads_unfair = Beta(concentration1=0.1, concentration0=0.1)  # green
pheads_unknown = Beta(concentration1=1.0, concentration0=1.0)  # red

In [ ]:
x = tf.range(0.0, 1.0, 0.001)
plt.plot(*sess.run([x, pheads_fair.prob(x)]));
plt.plot(*sess.run([x, pheads_unfair.prob(x)]));
plt.plot(*sess.run([x, pheads_unknown.prob(x)]));
plt.axvline(x=pheads_true);

In [ ]:
# FORWARD MODEL
pheads = pheads_unknown
c = Bernoulli(probs=pheads, sample_shape=N)

#### Exact Solution

In [ ]:
# INFERENCE
pheads_cond = ed.complete_conditional(pheads)
## inferences/conjugacy.py
##pheads_cond = pheads

## /anaconda3/lib/python3.7/site-packages/edward/util/random_variables.py
## line 234
pheads_post = ed.copy(pheads_cond, {c: c_train})

In [ ]:
help(ed.complete_conditional)

In [ ]:
sess.run({key: val for
          key, val in six.iteritems(pheads_post.parameters)
          if isinstance(val, tf.Tensor)})

In [ ]:
# CRITICISM
mean, stddev = sess.run([pheads_post.mean(), pheads_post.stddev()])
print("Exact posterior mean:")
print(mean)
print("Exact posterior std:")
print(stddev)

In [ ]:
x = tf.range(0.0, 1.0, 0.001)
plt.plot(*sess.run([x, pheads.prob(x)]));  # blue
plt.plot(*sess.run([x, pheads_post.prob(x)]));  # green
plt.axvline(x=pheads_true);  # blue

In [ ]:
# this can take a minute
fig = plt.figure()
ax = plt.axes(xlim=(-0.05, 1.05), ylim=(-1.0, 11.0))

def go(pheads_prior, sample_shape, c_train):
    # MODEL
    c = Bernoulli(probs=pheads_prior,
                  sample_shape=sample_shape)
    # INFERENCE
    pheads_cond = ed.complete_conditional(pheads_prior)
    pheads_post = ed.copy(pheads_cond, {c: c_train[:sample_shape]})
    
    # CRITICISM
    ax.plot(*sess.run([x, pheads_post.prob(x)]));
    
    # RECURSION
    if len(c_train[sample_shape:]) >= sample_shape:
        go(pheads_post, sample_shape, c_train[sample_shape:])

pheads_prior = Beta(concentration1=1.0, concentration0=1.0)
ax.plot(*sess.run([x, pheads_prior.prob(x)]));  # blue
plt.axvline(x=pheads_true);  # blue
go(pheads_prior, 33, c_train)

#### MCMC: Metropolis Hastings

In [ ]:
# BACKWARD MODEL
T = 10000  # number of empirical samples
q_pheads = Empirical(params=tf.Variable(tf.ones([T])*.5))

In [ ]:
# INFERENCE
proposal_pheads = Beta(concentration1=1.0,
                       concentration0=1.0)
inference = ed.MetropolisHastings(latent_vars={pheads: q_pheads},
                                  proposal_vars={pheads: proposal_pheads},
                                  data={c: c_train})
inference.run()

In [ ]:
# CRITICISM
mean, stddev = sess.run([q_pheads.mean(), q_pheads.stddev()])
print("Inferred posterior mean:")
print(mean)
print("Inferred posterior std:")
print(stddev)

In [ ]:
plt.plot(q_pheads.params.eval());
plt.axhline(y=pheads_true);

In [ ]:
def lags(x):
    mean = tf.reduce_mean(x)
    var = tf.cast(tf.size(x) - 1, tf.float32) * tf.reduce_mean(tf.square(x - mean))
    ret = tf.map_fn(lambda k: tf.cond(tf.equal(k, 0),
                                      lambda: var,
                                      lambda: tf.reduce_sum((x[:-k] - mean) * (x[k:] - mean))),
                    tf.range(0, tf.size(x)),
                    dtype=tf.float32)
    return ret / var

In [ ]:
plt.plot(lags(q_pheads.params).eval());

In [ ]:
x = tf.range(0.0, 1.0, 0.001)
plt.plot(*sess.run([x, pheads.prob(x)]));  # blue
plt.plot(*sess.run([x, pheads_cond.prob(x)],  # green
                   {c: c_train}));
plt.hist(q_pheads.params.eval(),  # red
         bins=100, range=(0.0, 1.0),
         normed=True);
plt.axvline(x=pheads_true);  # blue

#### Variational Inference (VI)

In [ ]:
# BACKWARD MODEL
q_pheads_concentration1 = tf.nn.softplus(tf.Variable(tf.random_normal([])))
# q_pheads_concentration1 = tf.nn.softplus(tf.Variable(51 + tf.random_normal([])))
q_pheads_concentration0 = tf.nn.softplus(tf.Variable(tf.random_normal([])))
# q_pheads_concentration0 = tf.nn.softplus(tf.Variable(51 + tf.random_normal([])))
q_pheads = Beta(concentration1=q_pheads_concentration1,
                concentration0=q_pheads_concentration0)

In [ ]:
x = tf.range(-5.0, 5.0, 0.001)
plt.plot(*sess.run([x, tf.nn.softplus(x)]));

In [ ]:
# T = 10000  # number of empirical samples
# q_pheads_samples = sess.run(q_pheads.sample(sample_shape=T))

In [ ]:
# plt.plot(q_pheads_samples);
# plt.axhline(y=pheads_true);

In [ ]:
sess.run({key: val for
          key, val in six.iteritems(q_pheads.parameters)
          if isinstance(val, tf.Tensor)})

In [ ]:
plt.plot(*sess.run([x, pheads.prob(x)]));  # blue
plt.plot(*sess.run([x, pheads_cond.prob(x)],  # green
                   {c: c_train}));
plt.plot(*sess.run([x, q_pheads.prob(x)]));  # red
# plt.hist(q_pheads_samples,  # red
#          bins=100, range=(0.0, 1.0),
#          normed=True);
plt.axvline(x=pheads_true);  # blue

In [ ]:
# CRITICISM
mean, stddev = sess.run([q_pheads.mean(), q_pheads.stddev()])
print("Inferred posterior mean:")
print(mean)
print("Inferred posterior std:")
print(stddev)